# Selecting columns 2: using `select` and expressions
By the end of this lecture you will be able to:
- select a column or columns with `select`
- transform a column while selecting it
- select a column in lazy mode

Selecting columns with expressions is key to performant analysis as:
- this approach works in lazy mode
- when we select and transform multiple columns Polars will run these selections in paralell

We introduce the range of methods we can use to select columns with an expression in this lecture.

In [1]:
import polars as pl

In [2]:
csv_file = "../data/titanic.csv"

In [3]:
df = pl.read_csv(csv_file)
df.head(3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S"""


## Selecting a single column with a string

We can choose a column with a string in the `select` method

In [6]:
(
    df
    .select('Age', 'Survived')
    .head(3)
)

Age,Survived
f64,i64
22.0,0
38.0,1
26.0,1


Note that the output of `select` is always a `DataFrame` rather than a `Series` even if just one column is selected.

We can use `to_series` if we want a `Series`

In [9]:
(
    df
    .select('Age')
    .to_series()
    .head(3)
)

Age
f64
22.0
38.0
26.0


### Selecting multiple columns

We can select multiple columns with comma-separated strings

In [ ]:
(
    df
    .select(
        'Survived','Age'
    )
    .head(3)
)

Or we can pass a list of column names to `select`

In [ ]:
(
    df
    .select(
        ['Survived','Age']
    )
    .head(3)
)

There is no performance difference between using comma-separated column names or a list of column names.

## Differences between using `select` and `[]`

- `[]` indexing can only be used in eager mode, but **`select` can also be used in lazy mode**
- expressions in `select` can be **optimised** in lazy mode by the query optimiser
- multiple expressions in `select` can be run in *parallel*

Get in the habit of using `select` as your default

## Selecting columns with an expression

We can select a column with an expression in the `select` method

In [10]:
(
    df
    .select(
        pl.col('Age', 'Survived')
    )
    .head(3)
)

Age,Survived
f64,i64
22.0,0
38.0,1
26.0,1


In practice when we pass a string instead of an expression (as above) polars uses that string to create an expression internally.

## Selecting and transforming a column with an expression
We can apply a transformation to a column before we output it.

In this example we use the `round` expression to round the values of the `Fare` column

In [ ]:
(
    df
    .select(
        pl.col('Fare').round(0)
    )
    .head(3)
)

In [11]:
(
    df
    .select(
        pl.col('Fare').round(0).alias('roundedFare')
    )
    .head(3)
)

roundedFare
f64
7.0
71.0
8.0


We will see many more examples where we use expressions to transform data as we go through the course.

### Selecting multiple columns with a list of expressions

We can also pass multiple expressions seperated by commas or in a `list`. 

In this case we use the `alias` expression to change the name of one column in the output

In [ ]:
(
    df
    .select(
        pl.col('Fare'),
        pl.col('Fare').round(0).alias('roundedFare')
    )
    .head(3)
)

Recall that when you have multiple expressions Polars runs them in parallel.

## Returning a single value
If you have a `DataFrame` with a single value - that is a `DataFrame` with shape `[1,1]` you can
return that value with square brackets or the `item` method

In [4]:
(
    pl.read_csv(csv_file)
    .select(
        pl.col('Name').first()
    )
    .item()
)

'Braund, Mr. Owen Harris'

In [13]:
(
    pl.read_csv(csv_file)
    .select(
        pl.col('Name')
    )
    .to_series()
    .to_list()
)


['Braund, Mr. Owen Harris',
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
 'Heikkinen, Miss. Laina',
 'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
 'Allen, Mr. William Henry',
 'Moran, Mr. James',
 'McCarthy, Mr. Timothy J',
 'Palsson, Master. Gosta Leonard',
 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
 'Nasser, Mrs. Nicholas (Adele Achem)',
 'Sandstrom, Miss. Marguerite Rut',
 'Bonnell, Miss. Elizabeth',
 'Saundercock, Mr. William Henry',
 'Andersson, Mr. Anders Johan',
 'Vestrom, Miss. Hulda Amanda Adolfina',
 'Hewlett, Mrs. (Mary D Kingcome) ',
 'Rice, Master. Eugene',
 'Williams, Mr. Charles Eugene',
 'Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)',
 'Masselmani, Mrs. Fatima',
 'Fynney, Mr. Joseph J',
 'Beesley, Mr. Lawrence',
 'McGowan, Miss. Anna "Annie"',
 'Sloper, Mr. William Thompson',
 'Palsson, Miss. Torborg Danira',
 'Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)',
 'Emir, Mr. Farred Chehab',
 'Fortune, Mr. Charles Alexander',
 '

## Selecting columns in lazy mode

If we apply `select` in lazy mode it changes the `PROJECT` part of the optimised query plan

In [15]:
df = (
    pl.scan_csv(csv_file)
    .select(['Survived','Age'])
)
print(df.explain())
print(df)

Csv SCAN [../data/titanic.csv]
PROJECT 2/12 COLUMNS
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 SELECT [col("Survived"), col("Age")] FROM
  Csv SCAN [../data/titanic.csv]
  PROJECT */12 COLUMNS


The optimized query plan now has:

`PROJECT 2/12 COLUMNS`

This means that Polars only loads the `Survived` and `Age` columns into memory when reading the CSV.

Reducing the number of columns reduces time and memory usage

## Exercises

In the exercises you will develop your understanding of:
- selecting columns using the `select` method
- transforming columns within the `select` method
- using `select` in lazy mode

### Exercise 1

Select the `Age` and `Survived` columns using the Expression API

Do this twice:
- once using strings
- once using expressions

In [ ]:
df = pl.read_csv(csv_file)
df.<blank>.head(3)
df.<blank>.head(3)

### Exercise 2
Select all rows where `Age` is greater than 30 and output the `Age` and `Survived` columns

In [ ]:
df = pl.read_csv(csv_file)
df.<blank>.head(3)

### Exercise 3
Output a one-column DataFrame where the column is the `min` of the `Age` column

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

Add another line onto the query to output this single value as a float

Output a one-row DataFrame where the first column is the `max` of the `Age` column and the second column is the `min` of the `Age` column

Expand the following cell if you want a hint

In [ ]:
#Hint: 
#you cannot have two columns with the same name so you will have to use the `alias` expression 

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

## Solutions

### Solution to Exercise 1
Select the `Age` and `Survived` columns using the Expression API

Do this twice:
- once using strings
- once using expressions

In [ ]:
df = pl.read_csv(csv_file)
df.select(['Age','Survived']).head(3)
df.select([pl.col('Age'),pl.col('Survived')]).head(3)

### Solution to Exercise 2
Select all rows where `Age` is greater than 30 and output the `Age` and `Survived` columns

In [ ]:
df = pl.read_csv(csv_file)
df.filter(pl.col('Age')>30).select(['Age','Survived']).head(3)

### Solution to Exercise 3
Output a one-column DataFrame where the column is the `min` of the `Age` column

In [ ]:
(
    pl.read_csv(csv_file)
    .select(
        pl.col('Age').min()
    )
)

Add another line onto the query to output this single value as a float

In [ ]:
(
    pl.read_csv(csv_file)
    .select(
        pl.col('Age').min()
    )
    .item()
)

Output a one-row DataFrame where the first column is the `max` of the `Age` column and the second column is the `min` of the `Age` column

In [16]:
(
    pl.read_csv(csv_file)
    .select(
        [
            pl.col('Age').max().alias('age_max'),
            pl.col('Age').min().alias('age_min')
        ]
    )
)

age_max,age_min
f64,f64
80.0,0.42


In [21]:
(
    pl.read_csv(csv_file)
    .filter(
        (pl.col('Age') == pl.col('Age').max())|(pl.col('Age') == pl.col('Age').min())
    )
    .select(
        pl.col('Age')
    )
)

Age
f64
80.0
0.42


In [23]:
(
    pl.read_csv(csv_file)
    .select(
        pl.col('Age').alias('age_max'),
        pl.col('Age').alias('age_min'),
    )
)

age_max,age_min
f64,f64
22.0,22.0
38.0,38.0
26.0,26.0
35.0,35.0
35.0,35.0
…,…
27.0,27.0
19.0,19.0
null,null
